In [15]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from bs4 import BeautifulSoup
import requests
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy

/mnt/c/Users/natha/Documents/TourdeFinance/trading_env/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [16]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [17]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [18]:
#use get_available_datasets() to see all the different Fama French datasets available
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [19]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1225,2015-04-17,-1.23,-0.43,0.23,0.13,0.10,0.000
1226,2015-04-16,-0.08,-0.10,-0.20,-0.23,-0.21,0.000
1227,2015-04-15,0.57,0.27,0.32,-0.19,-0.07,0.000
1228,2015-04-14,0.11,-0.13,0.19,0.09,0.09,0.000


In [20]:
# get sp500 tickers, code courtesy of Will Everett
page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(page.content, 'html.parser')
sp500_tickers = [row.find('td').get_text().replace('\n','')
                 for row in soup.find('tbody').find_all('tr')[1:]]
sp500_tickers.remove('BRK.B')
sp500_tickers.remove('BF.B')
sp500_tickers.sort()

In [21]:
days_of_data = 1000
open_close = pd.DataFrame()
for i in range(0,len(sp500_tickers),100) :
    data = api.get_barset(sp500_tickers[i:i+99],'day',limit=days_of_data)
    for ticker in sp500_tickers[i:i+99] :
        opin = []
        close = []
        for day in range(days_of_data - 1, -1, -1) :
            try :
                if (data[ticker][day].o == 0) :
                    opin.append(np.nan)
                else :
                    opin.append(data[ticker][day].o)
                close.append(data[ticker][day].c)
            except :
                opin.append(np.nan)
                close.append(np.nan)
        open_close[ticker + ' Open'] = opin
        open_close[ticker + ' Close'] = close

In [22]:
open_close

,A Open,A Close,AAL Open,AAL Close,AAP Open,AAP Close,AAPL Open,AAPL Close,ABBV Open,ABBV Close,...,XYL Open,XYL Close,YUM Open,YUM Close,ZBRA Open,ZBRA Close,ZION Open,ZION Close,ZTS Open,ZTS Close
0,76.43,78.2800,12.64,12.5100,104.62,106.98,268.70,267.960,78.56,79.7250,...,69.680,69.34,77.590,80.25,198.83,201.26,30.00,31.0000,126.55,128.750
1,74.08,76.6900,10.71,11.3300,102.26,102.48,263.10,266.040,76.44,78.5500,...,66.115,69.10,72.125,75.37,195.74,196.65,28.24,29.4300,125.15,127.170
2,76.61,74.0300,11.07,10.2175,102.23,99.71,270.80,259.385,78.38,75.2700,...,67.310,66.18,73.990,71.47,199.70,192.36,28.10,28.0400,129.02,126.170
3,72.97,74.2500,9.72,9.5000,89.44,95.80,250.90,262.285,75.74,75.6200,...,63.390,65.19,66.550,70.43,184.60,191.52,25.74,26.9200,119.58,126.740
4,71.71,70.4000,10.34,9.3900,86.14,84.76,242.80,241.430,74.65,73.3508,...,63.120,60.53,65.710,63.30,173.02,177.57,25.21,24.6800,115.08,114.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41.82,41.3000,36.79,36.7100,157.73,158.26,96.00,97.810,60.93,60.7100,...,42.290,42.21,81.750,81.53,64.02,64.10,27.13,27.3201,47.61,47.470
996,41.71,41.6300,36.57,36.9200,157.59,157.40,103.91,104.300,56.78,60.9200,...,41.960,42.46,81.500,82.02,65.18,63.66,26.74,27.2800,47.40,47.600
997,41.98,41.8000,37.46,36.3000,157.38,158.43,105.00,105.080,56.92,56.7800,...,42.170,41.83,81.180,81.34,64.46,65.25,26.98,26.7200,48.34,47.340
998,41.50,41.9300,36.96,37.4900,158.40,157.16,105.02,105.680,57.07,57.2000,...,42.710,42.17,81.650,81.24,64.59,64.70,26.42,26.9200,48.54,48.645


In [24]:
for tkr in sp500_tickers :
    try :
        open_close[tkr + ' Open'][0]
    except :
        sp500_tickers.remove(tkr)

In [25]:
len(sp500_tickers)

498

In [26]:
#number of days between today and when the stock market got hit by COVID-19
today = datetime.date.today()
days_since_crash = wkdy.networkdays(datetime.date(2020, 2, 19), today)
days_since_crash

38

In [27]:
hit_hard = []
for tkr in sp500_tickers :
    dif = (open_close[tkr + ' Open'][days_since_crash] -
           open_close[tkr + ' Close'][0]) / open_close[tkr + ' Open'][days_since_crash]
    hit_hard.append(dif)

In [29]:
len(hit_hard)

498

In [30]:
#finds the n greatest numbers in a list
def n_max(lis, n) :
    maxes = [] 
    for i in range(0, n):  
        max1 = 0
        for j in range(len(lis)):      
            if lis[j] >= max1: 
                try :
                    maxes.index(lis[j])
                except ValueError :
                    max1 = lis[j]
        maxes.append(max1)
    return maxes

In [31]:
discounted_ten_percent = n_max(hit_hard, 50)
discounted_indicies = []
for i in range(len(discounted_ten_percent)) :
    discounted_indicies.append(hit_hard.index(discounted_ten_percent[i]))
len(discounted_indicies)

50

In [35]:
portfolio_stocks = []
for index in discounted_indicies :
    portfolio_stocks.append(sp500_tickers[index])
portfolio_stocks

['IR',
 'NCLH',
 'CCL',
 'APA',
 'ARNC',
 'RCL',
 'OXY',
 'HAL',
 'MRO',
 'OKE',
 'ADS',
 'NBL',
 'UAL',
 'M',
 'DAL',
 'DVN',
 'AAL',
 'HP',
 'MPC',
 'BA',
 'KSS',
 'MGM',
 'FANG',
 'SLB',
 'ALK',
 'FTI',
 'SPG',
 'NOV',
 'GPS',
 'EXPE',
 'XRX',
 'DFS',
 'UNM',
 'SYF',
 'LYV',
 'TDG',
 'COTY',
 'DRI',
 'WYNN',
 'AIG',
 'PXD',
 'JWN',
 'TPR',
 'IVZ',
 'GE',
 'CMA',
 'MAR',
 'PVH',
 'LNC',
 'PHM']

In [36]:
#calculate the days between now and the fama french data
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
#presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today)

In [41]:
excess_returns = pd.DataFrame()
num_data = days_of_data - days_offset
excess_returns['Date'] = ff5['Date'][:num_data]
for tkr in portfolio_stocks :
    ex_returns = []
    for day in range(days_offset, days_of_data) :
        daily_ret = (open_close[tkr + ' Close'][day] -
                     open_close[tkr + ' Open'][day] -
                     ff5['RF'][day]) / open_close[tkr + ' Open'][day]
        ex_returns.append(daily_ret)
    excess_returns[tkr] = ex_returns

In [42]:
excess_returns

,Date,IR,NCLH,CCL,APA,ARNC,RCL,OXY,HAL,MRO,...,PXD,JWN,TPR,IVZ,GE,CMA,MAR,PVH,LNC,PHM
0,2020-02-28,-0.001841,-0.084580,-0.083660,-0.041804,0.000440,-0.089518,-0.057237,-0.043994,-0.067408,...,-0.041595,-0.017971,-0.043369,-0.030489,-0.036588,-0.026310,-0.048111,-0.044229,-0.029877,-0.014202
1,2020-02-27,-0.032424,-0.085355,-0.057000,-0.063363,-0.044759,-0.084537,-0.084876,-0.047419,-0.061684,...,-0.048881,-0.061840,-0.056434,-0.057957,-0.054804,-0.051545,-0.087477,-0.060398,-0.057293,-0.042756
2,2020-02-26,0.002427,-0.042841,-0.041036,-0.008383,0.020494,-0.033777,-0.025333,-0.020543,-0.018257,...,0.002019,-0.000026,-0.029852,-0.002143,0.014029,-0.017360,-0.014589,0.010249,-0.034962,-0.007807
3,2020-02-25,0.003404,-0.024231,-0.010116,0.008671,0.000120,-0.025230,-0.010247,-0.017837,-0.005517,...,0.008534,-0.001871,-0.016380,-0.022974,-0.016546,-0.007562,-0.014761,-0.014389,-0.007572,-0.006338
4,2020-02-24,0.005606,-0.058186,-0.002956,-0.000909,-0.012199,-0.009795,0.006066,-0.026021,-0.029953,...,-0.010319,0.014173,0.025053,0.012584,-0.010681,0.009777,-0.004718,0.010045,0.007497,0.003744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016-04-29,-0.024918,0.022777,0.007307,0.020184,-0.014581,0.014223,0.009492,0.029340,0.005356,...,0.037975,0.015376,0.008137,0.012141,0.000291,0.031749,0.024295,0.020681,-0.004795,-0.002704
965,2016-04-28,-0.000328,-0.005718,-0.059813,0.017836,-0.546303,0.013500,-0.001741,-0.000283,0.024769,...,0.006551,0.010076,0.034296,-0.010911,0.002562,-0.001624,-0.005950,-0.002838,0.013460,-0.009922
966,2016-04-27,0.011756,0.011882,0.009824,-0.022296,-0.024373,-0.035559,0.012804,-0.017569,-0.029544,...,0.009286,-0.014348,-0.006695,-0.001265,-0.005577,0.034524,0.003431,-0.007655,0.005520,0.000470
967,2016-04-26,-0.002488,-0.027209,-0.013446,0.016808,1.209492,-0.027994,-0.007725,0.016744,0.025698,...,-0.029075,-0.005989,-0.000273,-0.001886,-0.010968,-0.007033,-0.024784,0.002695,0.028296,0.028916


In [46]:
betas = pd.DataFrame()
X = pd.DataFrame(ff5[['Mkt-RF','SMB','HML','RMW','CMA']][0:num_data], columns=['Mkt-RF','SMB','HML','RMW','CMA'])
for tkr in portfolio_stocks :
    y = excess_returns[tkr]
    model = LinearRegression().fit(X, y)
    betas[tkr] = model.coef_

In [49]:
a = [1,2,3,4,5,6,6]
w = [.7,1,1,1,1,1,1.3]
np.average(a, weights = w)

4.071428571428571

In [69]:

weights = np.ones(len(ff5))
i = 1
for j in range(len(ff5)) :
    weights[j] *= i
    i -= .00005
weights

array([1.     , 0.99995, 0.9999 , ..., 0.9384 , 0.93835, 0.9383 ])

In [4]:
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
premiums = []
for factor in factors :
    av = np.average(ff5[factor])
    premiums.append(av)
premiums

[0.035162337662337666, -0.015673701298701298, -0.021566558441558442]

In [28]:
expected_returns = ff5['RF'][0]
for i in range(3) :
    expected_returns += betas[i]*premiums[i]
expected_returns

0.005980860984122728

In [3]:
#c of objective funct
#A_ub and b_ub
num_assets = 5
elem_vect = np.ones(num_assets)
#solution = opt.linprog(A_eq = elem_vect, b_eq = np.ones(1))

In [6]:
api.submit_order(symbol = 'AAPL',
                 qty = 3,
                 side = 'buy',
                 type = 'market',
                 time_in_force = 'day')

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'a192e8d2-b3b1-4a79-8054-dfd649e61a56',
    'created_at': '2020-04-03T21:32:04.784938Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '77439e17-4f58-401c-a38c-89029d04868c',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '3',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-03T21:32:04.777419Z',
    'symbol': 'AAPL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-03T21:32:04.784938Z'})

In [ ]:
clock = api.get_clock()
if clock.is_open :
    #trade away! 

In [7]:
ff5

{0:             Mkt-RF   SMB   HML   RMW   CMA     RF
 Date                                             
 2015-03-09    0.37  0.09 -0.02  0.26 -0.03  0.000
 2015-03-10   -1.63  0.43 -0.47  0.09 -0.14  0.000
 2015-03-11   -0.04  0.62  0.52 -0.47 -0.03  0.000
 2015-03-12    1.28  0.40  0.50 -0.06  0.23  0.000
 2015-03-13   -0.57  0.19 -0.04 -0.10 -0.13  0.000
 ...            ...   ...   ...   ...   ...    ...
 2020-01-27   -1.56  0.27 -0.48 -0.59  0.05  0.006
 2020-01-28    1.02 -0.15 -0.42  0.17 -0.10  0.006
 2020-01-29   -0.10 -0.44 -0.95  0.25  0.19  0.006
 2020-01-30    0.34 -0.64  0.67 -0.39 -0.11  0.006
 2020-01-31   -1.74 -0.52 -0.34 -0.54 -0.69  0.006
 
 [1235 rows x 6 columns],
 'DESCR': 'F-F Research Data 5 Factors 2x3 daily\n-------------------------------------\n\nThis file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202001 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.\n\n  0 : (1235 rows x 6 cols)'}